In [39]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

nb_name = '001'

# データの理解
EDA

In [34]:
data_path = '../datas/train.csv'
test_df = pd.read_csv(data_path, parse_dates=True, index_col='date')

In [17]:
datas = ['test', 'train', 'sample_submission']
for data in datas:
    data_path = f'../datas/{data}.csv'
    _data_doc = f'../docs/{nb_name}_{data}_doc.txt'
    _df = pd.read_csv(data_path)
    with open(_data_doc, 'w') as f:
        f.write('Data Description\n')
        f.write('----------\n')
        f.write(f'{data} data has {_df.shape} and {len(_df.columns)} columns.\n')
        f.write(f'{data} data columns: {_df.columns}\n')
        f.write(f'{data} data head: {_df.head()}\n')
        f.write('----------\n')
        for col in _df.columns:
            f.write(f'{col} has {_df[col].unique()}.\n')
            f.write(f'{col} has {_df[col].nunique()} unique values.\n')
            f.write('----------\n')

In [26]:
# 欠損値処理
nan_cols = test_df.columns[test_df.isnull().any()]
nan_index = test_df.index[test_df.loc[:, nan_cols].isnull().any(axis=1)]
print(f'nan columns: {nan_cols}')
print(f'nan index: {nan_index}, {len(nan_index)}')

nan columns: Index(['num_sold'], dtype='object')
nan index: Index([     0,     45,     90,    135,    180,    225,    270,    315,    320,
          360,
       ...
       229680, 229725, 229770, 229815, 229860, 229905, 229950, 229995, 230040,
       230085],
      dtype='int64', length=8871), 8871


In [18]:
test_df.head()

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [20]:
test_df.info()
test_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


,id,num_sold
count,230130.000000,221259.000000
mean,115064.500000,752.527382
std,66432.953062,690.165445
min,0.000000,5.000000
25%,57532.250000,219.000000
50%,115064.500000,605.000000
75%,172596.750000,1114.000000
max,230129.000000,5939.000000


In [48]:
# print(test_df['num_sold'].resample('10D').mean())
# print(test_df['num_sold'].resample('YE').sum())
intervals = ['D', '10D','W', 'ME', 'YE']
fig_fol = f'../results/'

for i, interval in enumerate(intervals):
    fig, axes = plt.subplots(len(test_df['store'].unique()), 1,figsize=(7, 5))
    for j, store in enumerate(test_df['store'].unique()): # storeごとの売り上げを確認
        for k, product in enumerate(test_df['product'].unique()):
            _plot_df = test_df[(test_df['store'] == store) & (test_df['product'] == product)]
            _means = _plot_df['num_sold'].resample(interval).mean()
            axes[j].plot(_means, label=product)
        axes[j].set_title(f'{store}')
    fig.suptitle(f'num_sold resample by {str(interval)}')
    plt.tight_layout()
    plt.legend()
    plt.savefig(f'{fig_fol}/{nb_name}_num_sold_resample_{interval}.png')
    # plt.show()
    plt.close()
    

In [31]:
def feature_engineering(df):
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['year'] = df['date'].dt.year
    df['weekday'] = df['date'].dt.weekday
    df['week'] = df['date'].dt.week
    df['quarter'] = df['date'].dt.quarter
    df['is_month_start'] = df['date'].dt.is_month_start
    df['is_month_end'] = df['date'].dt.is_month_end
    df['is_quarter_start'] = df['date'].dt.is_quarter_start
    df['is_quarter_end'] = df['date'].dt.is_quarter_end
    df['is_year_start'] = df['date'].dt.is_year_start
    df['is_year_end'] = df['date'].dt.is_year_end
    df.drop('date', axis=1, inplace=True)
    return df

modified_test_df = feature_engineering(test_df)
display(modified_test_df.head())

AttributeError: 'DatetimeProperties' object has no attribute 'week'